<a href="https://colab.research.google.com/github/himanshudas13/MovieRecommenderSystem/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
user_ratings=pd.read_csv('/content/drive/MyDrive/Collaborative filtering/songs-collaborative/songsDataset.csv')
user_ratings.head()

,'userID','songID','rating'
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5


In [8]:
# prompt: user_ratings.shape() get keep only first 100 rows

user_ratings = user_ratings.iloc[:100]
user_ratings.shape


(100, 3)

In [6]:
user_ratings.shape()

TypeError: 'tuple' object is not callable

In [9]:

user_ratings.columns = user_ratings.columns.str.replace("'", "")
user_ratings.columns

Index(['userID', 'songID', 'rating'], dtype='object')

In [34]:


user_ratings_pivot = user_ratings.pivot(index='userID', columns='songID', values='rating').fillna(0)
user_ratings_pivot.head()
# user_ratings_matrix = csr_matrix(user_ratings_pivot.values)


songID,1539,1666,2323,3342,7171,7522,8063,8637,10438,11219,...,131629,131919,132216,132685,133420,133967,134269,134626,134732,136507
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,...,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# prompt: user_ratings_pivot = user_ratings_pivot.iloc[:, :10]
# user_ratings_pivot.head()
# after this fill random values between 0 to 10 for all cases where rating is 0

import random
user_ratings_pivot = user_ratings_pivot.iloc[:, :10]
user_ratings_pivot = user_ratings_pivot.applymap(lambda x: random.randint(0, 10) if x == 0 else x)
user_ratings_pivot.head()


<ipython-input-36-03f214692314>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_ratings_pivot = user_ratings_pivot.applymap(lambda x: random.randint(0, 10) if x == 0 else x)


songID,1539,1666,2323,3342,7171,7522,8063,8637,10438,11219
userID,,,,,,,,,,
0,9.0,1.0,1.0,0.0,5.0,5.0,4.0,4.0,5.0,3.0
1,1.0,8.0,9.0,5.0,9.0,1.0,4.0,4.0,1.0,10.0
2,8.0,5.0,7.0,6.0,2.0,1.0,7.0,5.0,3.0,9.0
3,5.0,1.0,4.0,8.0,3.0,4.0,9.0,2.0,10.0,5.0
4,5.0,8.0,7.0,7.0,6.0,9.0,0.0,1.0,6.0,10.0


In [39]:


import random
user_listening_time = user_ratings_pivot.applymap(lambda x: random.randint(0, 100) if x != 0 else 0)
user_listening_time.head()


<ipython-input-39-e92206cc74da>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_listening_time = user_ratings_pivot.applymap(lambda x: random.randint(0, 100) if x != 0 else 0)


songID,1539,1666,2323,3342,7171,7522,8063,8637,10438,11219
userID,,,,,,,,,,
0,57,58,100,0,48,33,25,11,39,35
1,10,41,19,8,95,45,12,81,49,72
2,52,83,75,10,42,9,83,68,100,65
3,96,85,50,48,3,56,33,5,59,82
4,81,52,69,40,61,34,0,48,1,67


In [29]:
def standardise(row):
  new_row = (row - row.mean())/(row.max()-row.min())
  return new_row

user_ratings_std = user_ratings_pivot.apply(standardise)
user_ratings_std.head(20)

songID,1539,1666,2323,3342,7171,7522,8063,8637,10438,11219
userID,,,,,,,,,,
0,-0.622222,-0.133333,0.571429,-0.350000,-0.177778,-0.34,0.144444,-0.011111,-0.59,0.44
1,0.266667,-0.133333,-0.285714,0.316667,0.266667,-0.34,-0.077778,-0.455556,0.11,0.04
2,0.155556,0.200000,0.142857,-0.183333,-0.622222,-0.04,-0.300000,-0.455556,-0.29,-0.56
3,0.266667,-0.244444,0.428571,0.316667,0.155556,0.56,0.477778,0.100000,0.41,0.24
4,-0.177778,-0.133333,-0.428571,-0.516667,0.377778,-0.34,0.366667,0.211111,0.41,-0.06
5,0.377778,0.755556,-0.285714,-0.350000,-0.066667,0.46,-0.300000,0.322222,-0.39,0.44
6,-0.066667,-0.133333,-0.142857,0.150000,-0.177778,-0.44,0.477778,-0.122222,0.31,-0.46
7,-0.511111,-0.133333,0.428571,-0.016667,0.377778,0.46,-0.300000,-0.344444,-0.29,-0.36
8,0.377778,-0.022222,0.000000,0.483333,-0.288889,0.36,0.033333,0.544444,0.31,-0.06


In [30]:
user_similarity=cosine_similarity(user_ratings_std)
user_similarity_df=pd.DataFrame(user_similarity,index=user_ratings_pivot.index,columns=user_ratings_pivot.index)
user_similarity_df

userID,0,1,2,3,4,5,6,7,8,9
userID,,,,,,,,,,
0,1.000000,-0.393802,0.024059,-0.207144,-0.079663,-0.089222,-0.181865,0.241441,-0.549590,-0.089492
1,-0.393802,1.000000,-0.065928,-0.104522,0.072932,-0.328962,0.296085,-0.182296,-0.196547,0.370372
2,0.024059,-0.065928,1.000000,-0.467694,-0.462256,0.046688,0.122624,0.194352,-0.168228,-0.329155
3,-0.207144,-0.104522,-0.467694,1.000000,-0.123745,-0.164997,-0.039751,0.006498,0.546568,-0.468395
4,-0.079663,0.072932,-0.462256,-0.123745,1.000000,-0.201490,0.413993,-0.307137,-0.274870,0.141721
5,-0.089222,-0.328962,0.046688,-0.164997,-0.201490,1.000000,-0.703089,-0.201577,0.124301,0.202216
6,-0.181865,0.296085,0.122624,-0.039751,0.413993,-0.703089,1.000000,-0.300599,0.017908,-0.265412
7,0.241441,-0.182296,0.194352,0.006498,-0.307137,-0.201577,-0.300599,1.000000,-0.368425,-0.292281
8,-0.549590,-0.196547,-0.168228,0.546568,-0.274870,0.124301,0.017908,-0.368425,1.000000,-0.044675


In [31]:


def get_similar_movies(user_id, user_ratings_pivot):
    # Sort the movies for the user in descending order based on ratings
    similar_movies = user_ratings_pivot.loc[user_id].sort_values(ascending=False)[:3]
    return similar_movies

def get_similar_users(user_id, user_similarity_df, user_ratings_pivot):
    similar_score = user_similarity_df[user_id]
    similar_score = similar_score.sort_values(ascending=False)[:3]  # Slice the top 3 similar users

    similar_movies = []
    for user in similar_score.index:
        similar_movies.append(get_similar_movies(user, user_ratings_pivot))

    return similar_movies

# Assuming user_similarity_df and user_ratings_pivot are already defined and structured correctly
similar_movies = get_similar_users(5, user_similarity_df, user_ratings_pivot)
print(similar_movies)


[songID
1539     10.0
1666     10.0
11219    10.0
Name: 5, dtype: float64, songID
11219    9.0
7171     8.0
8637     6.0
Name: 9, dtype: float64, songID
1539     10.0
10438     9.0
8637      9.0
Name: 8, dtype: float64]
